# README

This Jupyter Notebook can be executed in one of two ways:

(1) set definition_only to True, and it will define Part_I(), Part_II(), ..., Part_XI(). You can then execute Part_All() in one shot at the end.

(2) set definition_only to False, in which case as the parts are defined, they will be executed.

Option (1) is better for experimentation. Option (2) is better for going through the flow. If you are reading this, I recommend the second option (2).

In [1]:
definition_only = False

# Global Settings

This controls the behavior of the code

In [2]:
PRESETS_100_100_original = {
    'filename': 'tabular-100-100-original.csv',
    'weekly_alpha': 1.00,
    'season_cliff': 1.00,
    'normalize': False
}

PRESETS_100_100_norm = {
    'filename': 'tabular-100-100-norm.csv',
    'weekly_alpha': 1.00,
    'season_cliff': 1.00,
    'normalize': True
}

PRESETS_095_050_norm = {
    'filename': 'tabular-095-050-norm.csv',
    'weekly_alpha': 0.95,
    'season_cliff': 0.50,
    'normalize': True
}

PRESETS_095_050_original = {
    'filename': 'tabular-095-050-original.csv',
    'weekly_alpha': 0.95,
    'season_cliff': 0.50,
    'normalize': False
}

In [3]:
GLOBAL = PRESETS_095_050_original  # the key

# Part I -- Five Fifty Eight

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import statistics
import math

In [5]:
full_to_canon_list = [
    ('Browns', 'CLE'), ('Chiefs', 'KC'), ('Steelers', 'PIT'),
    ('Lions', 'DET'), ('Titans', 'TEN'), ('Rams', 'LAR'),
    ('Patriots', 'NE'), ('Buccaneers', 'TB'), ('Panthers', 'CAR'),
    ('Texans', 'HOU'), ('Packers', 'GB'), ('Cardinals', 'ARI'),
    ('Jaguars', 'JAX'), ('Giants', 'NYG'), ('Commanders', 'WSH'),
    ('Raiders', 'LV'), ('Falcons', 'ATL'), ('Bears', 'CHI'),
    ('Bengals', 'CIN'), ('Colts', 'IND'), ('Dolphins', 'MIA'),
    ('Vikings', 'MIN'), ('Eagles', 'PHI'), ('Ravens', 'BAL'),
    ('Seahawks', 'SEA'), ('Broncos', 'DEN'), ('Jets', 'NYJ'),
    ('Chargers', 'LAC'), ('Cowboys', 'DAL'), ('Bills', 'BUF'),
    ('Saints', 'NO'), ('49ers', 'SF')
]
abbrev_to_canon_list = [
    ('OAK', 'LV'),  # Oakland Raiders --> Las Vegas Raiders
    ('SD', 'LAC'),  # San Diego Chargers --> Los Angeles Chargers
    ('STL', 'LAR'),  # St Louis Rams --> Los Angeles Rams 
]
full_to_canon_dict = { x: y for x, y in full_to_canon_list }
abbrev_to_canon_dict = { x: y for x, y in abbrev_to_canon_list }

In [6]:
def Part_I():
    global fivethirtyeight
    fivethirtyeight = pd.read_csv('data/538/538-elo.csv')
    print(fivethirtyeight.columns)
    fivethirtyeight = pd.DataFrame(fivethirtyeight[fivethirtyeight['season'] >= 2006])
    fivethirtyeight = fivethirtyeight.reset_index(drop=True)
    fivethirtyeight['team1'].replace(abbrev_to_canon_dict, inplace=True)
    fivethirtyeight['team2'].replace(abbrev_to_canon_dict, inplace=True)
    fivethirtyeight.insert(6, 'rivalry', fivethirtyeight['team1'] + '-' + fivethirtyeight['team2'])
    fivethirtyeight = fivethirtyeight.sort_values(['date', 'rivalry'])
    fivethirtyeight.reset_index(drop=True)
    league = fivethirtyeight['team1'].unique()
    assert(len(league) == 32)  # 32-team league
    print(league)

if not definition_only:
    Part_I()

Index(['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre',
       'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
       'qbelo1_pre', 'qbelo2_pre', 'qb1', 'qb2', 'qb1_value_pre',
       'qb2_value_pre', 'qb1_adj', 'qb2_adj', 'qbelo_prob1', 'qbelo_prob2',
       'qb1_game_value', 'qb2_game_value', 'qb1_value_post', 'qb2_value_post',
       'qbelo1_post', 'qbelo2_post', 'score1', 'score2', 'quality',
       'importance', 'total_rating'],
      dtype='object')
['PIT' 'ARI' 'CAR' 'CLE' 'DET' 'GB' 'HOU' 'JAX' 'KC' 'LAR' 'NE' 'NYG' 'TB'
 'TEN' 'LV' 'WSH' 'ATL' 'BAL' 'CHI' 'CIN' 'DAL' 'DEN' 'IND' 'LAC' 'MIA'
 'MIN' 'NYJ' 'PHI' 'SEA' 'SF' 'BUF' 'NO']


# Part II -- ESPN NFL Team Stat (Kaggle Dataset)

Our data from ESPN is missing the following games (a data error on the ESPN side):

> 2007-12-30,DAL-WSH

> 2010-12-23,CAR-PIT

> 2012-01-01,ATL-TB

We choose to just delete them instead of recreating them. With this correction, the Kaggle dataset is consistent with the ESPN dataset, with 4573 entries (out of 4576) entries from 2006 to 2023.

In [7]:
def Part_II():
    global team_stat
    team_stat = pd.read_csv('data/kaggle/team-stat.csv')
    team_stat = pd.DataFrame(team_stat[team_stat['date'] > '2006-06-30'])  # 2002 season
    team_stat['home'].replace(full_to_canon_dict, inplace=True)
    team_stat['away'].replace(full_to_canon_dict, inplace=True)
    team_stat.insert(3, 'rivalry', team_stat['home'] + '-' + team_stat['away'])
    team_stat = team_stat.sort_values(['date', 'rivalry'])
    team_stat = team_stat.reset_index(drop=True)
    
    def colon_to_seconds(s):  # 34:56 --> 34 * 60 + 56
        return 60 * int(s.split(':')[0]) + int(s.split(':')[1])
    
    team_stat['possession_away'] = team_stat['possession_away'].apply(colon_to_seconds)
    team_stat['possession_home'] = team_stat['possession_home'].apply(colon_to_seconds)
    print(team_stat.columns)
    
if not definition_only:
    Part_II()

Index(['date', 'away', 'home', 'rivalry', 'first_downs_away',
       'first_downs_home', 'third_downs_away', 'third_downs_home',
       'fourth_downs_away', 'fourth_downs_home', 'passing_yards_away',
       'passing_yards_home', 'rushing_yards_away', 'rushing_yards_home',
       'total_yards_away', 'total_yards_home', 'comp_att_away',
       'comp_att_home', 'sacks_away', 'sacks_home', 'rushing_attempts_away',
       'rushing_attempts_home', 'fumbles_away', 'fumbles_home', 'int_away',
       'int_home', 'turnovers_away', 'turnovers_home', 'penalties_away',
       'penalties_home', 'redzone_away', 'redzone_home', 'drives_away',
       'drives_home', 'def_st_td_away', 'def_st_td_home', 'possession_away',
       'possession_home', 'score_away', 'score_home'],
      dtype='object')


# Part III -- More Lookup Tables

In [8]:
def Part_III():

    global homes_dict, aways_dict, team_dates_dict
    
    # generate homes_dict['PITT|2006-09-07'] --> index into the actual game
    # generate aways_dict['MIA|2006-09-07'] --> index into the actual game

    games = team_stat[['date', 'rivalry']]

    homes_dict = {}
    aways_dict = {}
    for index, r in games.iterrows():
        date = r['date']
        rivals = r['rivalry'].split('-')
        homes_dict[f'{rivals[0]}|{date}'] = index
        aways_dict[f'{rivals[1]}|{date}'] = index
        
    # generate team_dates_dict['PITT'] --> a list of dates when the team played

    team_dates_dict = {}
    for index, r in games.iterrows():
        date = r['date']
        rivals = r['rivalry'].split('-')
        for t in range(0, 2):
            homeaway = 'home' if t == 0 else 'away'
            entry = (date, rivals[1-t], homeaway)
            if rivals[t] not in team_dates_dict.keys():
                team_dates_dict[rivals[t]] = [entry]
            else:
                team_dates_dict[rivals[t]].append(entry)
                
if not definition_only:
    Part_III()

# Part IV -- ESPN QBR

In [9]:
def Part_IV():
    
    global espn_qbr_2006_2020, espn_qbr_2021, espn_qbr_2022
    
    espn_qbr_2006_2020 = pd.read_csv('data/espn/espn-qbr-nfl-weekly-2006-2020.csv')
    espn_qbr_2021 = pd.read_csv('data/espn/espn-qbr-nfl-weekly-2021.csv')
    espn_qbr_2022 = pd.read_csv('data/espn/espn-qbr-nfl-weekly-2022.csv')
    espn_qbr_2021 = espn_qbr_2021.drop(['Unnamed: 0'], axis=1)
    espn_qbr_2022 = espn_qbr_2022.drop(['Unnamed: 0'], axis=1)
    espn_qbr_2006_2020['team_abb'].replace(abbrev_to_canon_dict, inplace=True)
    espn_qbr_2006_2020['opp_abb'].replace(abbrev_to_canon_dict, inplace=True)
    espn_qbr_2021['team_abb'].replace(abbrev_to_canon_dict, inplace=True)
    espn_qbr_2021['opp_abb'].replace(abbrev_to_canon_dict, inplace=True)
    espn_qbr_2022['team_abb'].replace(abbrev_to_canon_dict, inplace=True)
    espn_qbr_2022['opp_abb'].replace(abbrev_to_canon_dict, inplace=True)
    
if not definition_only:
    Part_IV()

# Part V -- Calender Management

In [10]:
season_kickoffs = {}
season_first_last = pd.read_csv('data/hardy/season_first_last.csv')
for index, r in season_first_last.iterrows():
    first_day = datetime.strptime(r['first'], "%Y-%m-%d")
    last_day = datetime.strptime(r['last'], "%Y-%m-%d")  # superbowl
    delta_days = last_day - first_day + timedelta(days=1)
    season_kickoffs[first_day.year] = first_day

def parse_date(date_text):
    date = datetime.strptime(date_text, "%Y-%m-%d")
    season = date.year
    if date.month < 6:
        season -= 1  # NFL season
    kickoff = season_kickoffs[season]
    delta_days = (date - kickoff).days
    week = 1 + (delta_days // 7)
    regular_week_count = (17 + 1) if season >= 2021 else (16 + 1)
        # 17 games since 2021
        
    days_since_1900 = (date - datetime(1900, 1, 1)).days
    regular = week >= 1 and week <= regular_week_count
    if regular:
        week_name = f"Week {week}"
    else:
        if week - regular_week_count == 1:
            week_name = 'Wild Card'
        elif week - regular_week_count == 2:
            week_name = 'Divisional Round'
        elif week - regular_week_count == 3:
            week_name = 'Conference Championship'
        elif week - regular_week_count == 5:
            week_name = 'Super Bowl'
        else:
            week_name = 'Unknown'
    
    return {
        'date': date_text,
        'season': season,
        'week': week,
        'weekday': date.weekday(),
        'weekday-name': date.strftime("%A"),
        'week-name': week_name,
        'regular': regular,  # regular season
        'absolute': days_since_1900,
    }

season_first_last = pd.read_csv('data/hardy/season_first_last.csv')
for index, r in season_first_last.iterrows():
    superbowl = parse_date(r['last'])  # superbowl
    print(superbowl)
    
def Part_V():
    ()  # we should always do this

{'date': '2007-02-04', 'season': 2006, 'week': 22, 'weekday': 6, 'weekday-name': 'Sunday', 'week-name': 'Super Bowl', 'regular': False, 'absolute': 39115}
{'date': '2008-02-03', 'season': 2007, 'week': 22, 'weekday': 6, 'weekday-name': 'Sunday', 'week-name': 'Super Bowl', 'regular': False, 'absolute': 39479}
{'date': '2009-02-01', 'season': 2008, 'week': 22, 'weekday': 6, 'weekday-name': 'Sunday', 'week-name': 'Super Bowl', 'regular': False, 'absolute': 39843}
{'date': '2010-02-07', 'season': 2009, 'week': 22, 'weekday': 6, 'weekday-name': 'Sunday', 'week-name': 'Super Bowl', 'regular': False, 'absolute': 40214}
{'date': '2011-02-06', 'season': 2010, 'week': 22, 'weekday': 6, 'weekday-name': 'Sunday', 'week-name': 'Super Bowl', 'regular': False, 'absolute': 40578}
{'date': '2012-02-05', 'season': 2011, 'week': 22, 'weekday': 6, 'weekday-name': 'Sunday', 'week-name': 'Super Bowl', 'regular': False, 'absolute': 40942}
{'date': '2013-02-03', 'season': 2012, 'week': 22, 'weekday': 6, 'week

# Part VI -- Fixing ESPN QBR

The team associated with the QB is misattributed (it was the current team of the player,
not the team of the player at the time of the game), so we'll reverse-engineer the
actual team based on the opposing team and the date of the game

In [11]:
# the team associated with the QB is misattributed (it was the current team of the player,
# not the team of the player at the time of the game), so we'll reverse-engineer the
# actual team based on the opposing team and the date of the game

def fix_espn(espn_qbr):
    if 'name_display' in espn_qbr.keys():
        name_key = 'name_display'  # 2021, 2022
    else:
        name_key = 'name'  # 2006-2020
    for index, r in espn_qbr.iterrows():
        name = r[name_key]
        opp = r['opp_abb']
        team = r['team_abb']
        season = r['season']
        week_text = r['week_text']
        actual_team = None
        for (date, opp, homeaway) in team_dates_dict[opp]:
            parsed = parse_date(date)
            if parsed['season'] == season and parsed['week-name'] == week_text:
                assert(actual_team == None)
                actual_team = opp

        if actual_team == None:
            print(f'Unable to find {name}, {opp}, {team}, {r["season"]}, {r["week_text"]} (SKIPPED)')
            continue  # this happens because of the result of the three missing games

        assert(actual_team != None)
        if actual_team != team:
            espn_qbr.at[index, 'team_abb'] = actual_team
            print(f'{name} @{season}/{week_text} {team} (mistake) --> {actual_team} (correction)')
    return espn_qbr

In [12]:
def make_qbr_dict(verbose=False):
    espn_qbr_dict = {}
    for espn_qbr in [espn_qbr_2006_2020, espn_qbr_2021, espn_qbr_2022]:
    # for espn_qbr in [espn_qbr_2021]:
        grouped = espn_qbr.groupby(['season', 'season_type', 'game_week', 'week_text', 'team_abb'])
        for (season, season_type, game_week, week_text, team), group in grouped:
            mean_qbr_total = group['qbr_total'].mean()
            count_qbr_total = group['qbr_total'].count()
            matched_parsed = None
            for team_date in team_dates_dict[team]:
                parsed = parse_date(team_date[0])
                if parsed['season'] == season and parsed['week-name'] == week_text:
                    matched_parsed = parsed
            if matched_parsed == None:
                print(f'{team}: failed to match {season}, {week_text}')
                continue
            key = f'{team}|{matched_parsed["date"]}'
            espn_qbr_dict[key] = mean_qbr_total

            if verbose and count_qbr_total > 1:
                print(f"{team} {week_text} of {season}: QBR = {mean_qbr_total} ({count_qbr_total})")
    return espn_qbr_dict

In [13]:
def Part_VI():
    
    global espn_qbr_2006_2020, espn_qbr_2021, espn_qbr_2022, espn_qbr_dict
    
    fix_espn(espn_qbr_2006_2020)
    fix_espn(espn_qbr_2021)
    fix_espn(espn_qbr_2022)
    espn_qbr_dict = make_qbr_dict()
    
if not definition_only:
    Part_VI()

Unable to find Todd Collins, SF, WSH, 2007, Week 17 (SKIPPED)
Unable to find Tony Romo, DAL, DAL, 2007, Week 17 (SKIPPED)
Trent Edwards @2010/Week 1 JAX (mistake) --> BUF (correction)
Trent Edwards @2010/Week 2 JAX (mistake) --> BUF (correction)
Unable to find Ben Roethlisberger, NO, PIT, 2010, Week 16 (SKIPPED)
Unable to find Jimmy Clausen, CLE, CAR, 2010, Week 16 (SKIPPED)
Kyle Orton @2011/Week 1 KC (mistake) --> DEN (correction)
Kyle Orton @2011/Week 2 KC (mistake) --> DEN (correction)
Kyle Orton @2011/Week 3 KC (mistake) --> DEN (correction)
Kyle Orton @2011/Week 4 KC (mistake) --> DEN (correction)
Unable to find Josh Freeman, TB, TB, 2011, Week 17 (SKIPPED)
Brian Hoyer @2012/Week 16 PIT (mistake) --> ARI (correction)
Brian Hoyer @2012/Week 17 PIT (mistake) --> ARI (correction)
Josh Freeman @2013/Week 7 TB (mistake) --> MIN (correction)
Matt Flynn @2013/Week 12 LV (mistake) --> GB (correction)
Matt Flynn @2013/Week 13 LV (mistake) --> GB (correction)
Matt Flynn @2013/Week 14 LV (mi

Mike Glennon @2020/Week 17 NYG (mistake) --> JAX (correction)
Sam Darnold @2020/Week 17 CAR (mistake) --> NYJ (correction)
Jared Goff @2020/Wild Card DET (mistake) --> LAR (correction)
Mitchell Trubisky @2020/Wild Card BUF (mistake) --> CHI (correction)
Jared Goff @2020/Divisional Round DET (mistake) --> LAR (correction)
Josh Johnson @2021/Week 9 BAL (mistake) --> NYJ (correction)
Marcus Mariota @2022/Week 1 PHI (mistake) --> ATL (correction)
Baker Mayfield @2022/Week 1 TB (mistake) --> CAR (correction)
Jacoby Brissett @2022/Week 1 WSH (mistake) --> CLE (correction)
Derek Carr @2022/Week 1 NO (mistake) --> LV (correction)
Aaron Rodgers @2022/Week 1 NYJ (mistake) --> GB (correction)
Jacoby Brissett @2022/Week 2 WSH (mistake) --> CLE (correction)
Jimmy Garoppolo @2022/Week 2 LV (mistake) --> SF (correction)
Derek Carr @2022/Week 2 NO (mistake) --> LV (correction)
Marcus Mariota @2022/Week 2 PHI (mistake) --> ATL (correction)
Aaron Rodgers @2022/Week 2 NYJ (mistake) --> GB (correction)
Ba

# Part VII -- Five Thirty Eight ELO

In [14]:
def Part_VII():
    
    global elo_dict
    
    elo_dict = {}
    for index, r in fivethirtyeight.iterrows():
        home = r['team1']
        away = r['team2']
        date = r['date']
        for homeQ in [True, False]:
            homeaway = '1' if homeQ else '2'
            awayhome = '2' if homeQ else '1'
            home = r[f'team{homeaway}']
            key = f'{home}|{date}'
            elo_dict[key] = {
                'elo-post': r[f'elo{homeaway}_post'],
                'qb-elo-post': r[f'qbelo{homeaway}_post'],
                    # after the game (note, we should not use the -pre value before
                    # the game because it leaks details about who played the game
            }
            
if not definition_only:
    Part_VII()

# Part VIII -- Compute TwoHalves

Note that a game is made of two teams (obviously), called a rivalry, and the statistics are of the form "total_yards_home" and "total_yards_away". This is to separate the statistics into another dataframe with (roughly) twice the number of rows but half the number of columns. I call it "twohalves".

In [15]:
def Part_VIII():
    
    global twohalves_dict
    
    rows = []
    missing_qbr_count = 0
    for index, r in tqdm(team_stat.iterrows(), total=len(team_stat), desc='Processing rows (VIII)'):
        row_home = None
        row_away = None
        for homeQ in [True, False]:
            date = r['date']
            homeaway = 'home' if homeQ else 'away'
            awayhome = 'away' if homeQ else 'home'
            team = r[homeaway]
            opponent = r[awayhome]
            rivalry = f'{r[homeaway]}-{r[awayhome]}'
            first_downs = r[f'first_downs_{homeaway}']
            third_downs_raw = r[f'third_downs_{homeaway}']
            fourth_downs_raw = r[f'fourth_downs_{homeaway}']
            passing_yards = r[f'passing_yards_{homeaway}']
            rushing_yards = r[f'rushing_yards_{homeaway}']
            total_yards = r[f'total_yards_{homeaway}']
            passing_yard_ratio = passing_yards / max(1, total_yards)
            rushing_yard_ratio = rushing_yards / max(1, total_yards)
            comp_att = r[f'comp_att_{homeaway}']
            sacks_raw = r[f'sacks_{homeaway}']
            sack_count = int(sacks_raw.split('-')[0])
            sack_yards = int(sacks_raw.split('-')[1])
            rushing_attempts = r[f'rushing_attempts_{homeaway}']
            fumbles = r[f'fumbles_{homeaway}']
            ints = r[f'int_{homeaway}']
            turnovers = r[f'turnovers_{homeaway}']
            penalties_raw = r[f'penalties_{homeaway}']
            penalty_count = int(penalties_raw.split('-')[0])
            penalty_yards = int(penalties_raw.split('-')[1])
            redzone_raw = r[f'redzone_{homeaway}']
            redzone_count = int(redzone_raw.split('-')[0])
            redzone_yards = int(redzone_raw.split('-')[1])
            drives = r[f'drives_{homeaway}']
            def_st_td = r[f'def_st_td_{homeaway}']
            possession = r[f'possession_{homeaway}']
            score = int(r[f'score_{homeaway}'])
            score_opponent = int(r[f'score_{awayhome}'])
            wintieloss = 1 if score > score_opponent else (0 if score < score_opponent else 0.5)
            third_down_conversions = 0 if int(third_downs_raw.split('-')[1]) == 0 else \
                int(third_downs_raw.split('-')[0]) / int(third_downs_raw.split('-')[1])
                    # fixme (None rather than 0 is preferred but not handled correctly)
            third_downs = int(third_downs_raw.split('-')[1])
            fourth_down_conversions = 0 if int(fourth_downs_raw.split('-')[1]) == 0 else \
                int(fourth_downs_raw.split('-')[0]) / int(fourth_downs_raw.split('-')[1])
                     # fixme (None rather than 0 is preferred but not handled correctly)
            fourth_downs = int(fourth_downs_raw.split('-')[1])
            rushing_to_passing = int(rushing_yards) / max(1, int(passing_yards))
            comp = int(comp_att.split('-')[0])
            att = int(comp_att.split('-')[1])
            comp_att_ratio = comp / max(1, att)
            elo = elo_dict[f'{team}|{date}']['elo-post']
            qb_elo = elo_dict[f'{team}|{date}']['qb-elo-post']

            espn_qbr_key = f'{team}|{date}'
            if espn_qbr_key in espn_qbr_dict.keys():
                espn_qbr = espn_qbr_dict[espn_qbr_key]
                espn_qbr_missing = 0
            else:
                # print(f'something wrong with {espn_qbr_key}')
                missing_qbr_count += 1
                espn_qbr = 50    
                espn_qbr_missing = 1

            row = {
                'date': date,
                'team': team,
                'opponent': opponent,
                'rivalry': rivalry,
                'home?': homeQ,
                'first-downs': first_downs,
                'third-downs': third_downs,
                'third-down-conversion': third_down_conversions,
                'fourth-downs': fourth_downs,
                'fourth-down-conversion': fourth_down_conversions,
                'passing-yards': passing_yards,
                'rushing-yards': rushing_yards,
                'total-yards': total_yards,
                'rushing-to-passing': rushing_to_passing,
                'passing-yard-ratio': passing_yard_ratio,
                'rushing-yard-ratio': rushing_yard_ratio,
                'elo': elo,
                'qb-elo': qb_elo,
                'espn-qbr': espn_qbr,
                'espn-qbr-missing': espn_qbr_missing,
                'comp': comp,
                'att': att,
                'comp-att': comp_att_ratio,
                'sack_count': sack_count,
                'sack_yards': sack_yards,
                'rushing-attempts': rushing_attempts,
                'fumbles': fumbles,
                'int': ints,
                'turnovers': turnovers,
                'penalty_count': penalty_count,
                'penalty_yards': penalty_yards,
                'redzone_count': redzone_count,
                'redzone_yards': redzone_yards,
                'drives': drives,
                'def-st-td': def_st_td,
                'possession': possession,
                'score': score,
                'wintieloss': wintieloss,
            }
            if homeQ:
                row_home = row
            else:
                row_away = row

        # compute advantages, which is the ratio of a selection of numbers between the two teams

        #advantages = [
        #    'first-downs', 'third-down-conversion', 'fourth-down-conversion',
        #    'passing-yards', 'rushing-yards', 'total-yards', 'elo', 'comp-att',
        #    'sack_count', 'fum'
        #             ]

        rows.append(row_home)
        rows.append(row_away)

    missing_qbr_percent = missing_qbr_count / (2 * len(team_stat)) * 100
    print(f'missing {missing_qbr_count} QBRs ({missing_qbr_percent:.2f}%)')
    twohalves = pd.DataFrame(rows)
    
    twohalves_dict = dict(
        [(f"{e['team']}|{e['date']}", e) for e in twohalves.to_dict(orient='records')])
    
    teams = list(team_dates_dict.keys())
    for team in teams:
        count = 0
        team_dates = team_dates_dict[team]
        for team_date in team_dates:
            count += 1
        print(f'{team} has played a total of {count} games')
        
if not definition_only:
    Part_VIII()

Processing rows (VIII): 100%|█████████████| 4573/4573 [00:01<00:00, 2708.61it/s]


missing 552 QBRs (6.04%)
PIT has played a total of 290 games
MIA has played a total of 277 games
ARI has played a total of 284 games
SF has played a total of 291 games
CAR has played a total of 281 games
ATL has played a total of 283 games
CLE has played a total of 276 games
NO has played a total of 291 games
DET has played a total of 277 games
SEA has played a total of 297 games
GB has played a total of 297 games
CHI has played a total of 281 games
HOU has played a total of 284 games
PHI has played a total of 292 games
JAX has played a total of 281 games
DAL has played a total of 285 games
KC has played a total of 295 games
CIN has played a total of 286 games
LAR has played a total of 284 games
DEN has played a total of 284 games
NE has played a total of 305 games
BUF has played a total of 282 games
NYG has played a total of 287 games
IND has played a total of 293 games
TB has played a total of 281 games
BAL has played a total of 295 games
TEN has played a total of 283 games
NYJ has p

# Part IX -- Compute League Averages (Weekly)

Operating with twohalves, it is easy to compute the averages. The following include a pretty robust infrastructure that allows me to compute all kinds of averages.

In [16]:
def compute_league_averages():
    
    league = team_stat['home'].unique()  # all teams
    excluded_keys = ['date', 'team', 'opponent', 'rivalry', 'home?']
    league_averages = {}
    for index, r in tqdm(team_stat.iterrows(), total=len(team_stat), desc='Processing rows (IX)'):
        date = r['date']
        data = {}
        for team in league:
            key = f'{team}|{date}'
            if key in twohalves_dict.keys():
                entry = twohalves_dict[key]
                for k, v in entry.items():
                    if k not in excluded_keys:
                        if k not in data.keys():
                            data[k] = [v]
                        else:
                            data[k].append(v)
                        
        stat = {}
        for k, v in data.items():
            mean = statistics.mean(data[k])
            std = statistics.stdev(data[k])
            stat[k] = { 'mean': mean, 'std': std }
                    
        league_averages[date] = stat
    
    return league_averages

In [17]:
def compute_rolling_league_averages(data, window=8):
    dates = list(data.keys())
    rolling_league_averages = {}
    for index in tqdm(range(len(dates)), desc='Processing rows (IX)'):
        date = dates[index]
        row = data[date]
        current_date = parse_date(date)
        sum = {}
        for back_index in range(index, -1, -1):  # inclusive of current date
            back_date = parse_date(dates[back_index])
            if current_date['absolute'] - back_date['absolute'] > 366:
                break
            for k, v in data[back_date['date']].items():
                if k in sum.keys():
                    sum[k].append(v) 
                else:
                    sum[k] = [v]
        
        
        averages = {}
        for k in sum.keys():
            entries = sum[k]
            sum_means = 0
            sum_variances = 0
            for e in entries:
                sum_means += e['mean']
                sum_variances += e['std'] * e['std']
            overall_mean = sum_means / len(entries)
            overall_std = math.sqrt(sum_variances / len(entries))
                # this is an approximation, but not too far off
            averages[k] = { 'mean': overall_mean, 'std': overall_std }
            
        rolling_league_averages[date] = averages
        
    return rolling_league_averages

In [18]:
def Part_IX():
    
    global league_averages, rolling_league_averages
    
    league_averages = compute_league_averages()
    rolling_league_averages = compute_rolling_league_averages(league_averages)
    
if not definition_only:
    Part_IX()

Processing rows (IX): 100%|██████████████████| 915/915 [00:01<00:00, 504.51it/s]


# Part X -- Compute Averages

Operating with twohalves, it is easy to compute the averages. The following include a pretty robust infrastructure that allows me to compute all kinds of averages.

In [19]:
def compute_averages(method, weekly_alpha, season_cliff, normalize):
    # season so far, last season, last 365 days
    
    averageable_keys = list(next(iter(twohalves_dict.values())).keys())
    excluded_keys = ['date', 'team', 'opponent', 'rivalry', 'home?']
    averageable_keys = set([x for x in averageable_keys if x not in excluded_keys])
    print(averageable_keys)    

    if weekly_alpha == None:
        weekly_alpha = 1
    if season_cliff == None:
        season_cliff = 1
        
    assert(method in ['this-season', 'last-season', 'last-365', 'historical', 'last-game'])
    averages_dict = {}
    teams = list(team_dates_dict.keys())
    for team in tqdm(teams, desc='Processing rows (X)'):
        team_dates = team_dates_dict[team]
        for index in range(len(team_dates)):
            current_date = parse_date(team_dates[index][0])
            current_absolute = current_date['absolute']
            current_key = f'{team}|{current_date["date"]}'
            current_entry = twohalves_dict[current_key]  # just basic info
            current_head = { k: current_entry[k] for k in excluded_keys }
            cumu_count = 0
            cumu_sum = 0
            cumu_weight = 0
            cumu = None
            for back_index in range(index - 1, -1, -1):
                # going backward starting from index - 1, to 0 inclusive
                back_date = parse_date(team_dates[back_index][0])
                back_absolute = back_date['absolute']
                if method == 'this-season':
                    if back_date['season'] != current_date['season']:
                        break
                elif method == 'last-season':
                    if back_date['season'] == current_date['season']:
                        continue  # skip this season
                    elif back_date['season'] < current_date['season'] - 1:
                        break
                elif method == 'last-365':
                    if current_date['absolute'] - back_date['absolute'] > 366:  # generous
                        break
                elif method == 'historical':
                    ()  # okay
                elif method == 'last-game':  # take only the last game
                    if back_index != index - 1:
                        break
                else:
                    assert(False)  # unknown method

                season_adjustment = season_cliff ** (current_date['season'] - back_date['season'])
                    
                delta_week = (current_date['absolute'] - back_date['absolute']) / 7
                weight = (weekly_alpha ** delta_week) * season_adjustment
                back_key = f'{team}|{back_date["date"]}'
                back_entry = twohalves_dict[back_key]
                
                if normalize:
                    mu_sigma = rolling_league_averages[back_date['date']]
                    mu = { k: v['mean'] for k, v in mu_sigma.items() }
                    sigma = { k: v['std'] for k, v in mu_sigma.items() }
                    back_entry = {
                        k: (v - mu[k]) / max(1e-9, sigma[k]) if k in mu_sigma.keys() else v
                            for k, v in back_entry.items()
                    }
                
                if cumu_count == 0:
                    cumu = { k: weight * back_entry[k] for k in averageable_keys }
                else:
                    cumu = { k: cumu[k] + weight * back_entry[k] for k in averageable_keys }
                cumu_count += 1
                cumu_weight += weight
            if cumu_weight > 0:
                averages = { k: cumu[k] / cumu_weight for k in cumu }
            else:
                averages = { k: 0 for k in averageable_keys }
                                
            current_tail = { k: averages[k] for k in averageable_keys }
            averages_dict[current_key] = { **current_head, **current_tail }
                # this is to preserve the order of the keys for ease of debugging
            
    return averages_dict

In [20]:
def Part_X():

    global this_season, last_season, historical, last_365, last_game
    
    weekly_alpha = GLOBAL['weekly_alpha']
    season_cliff = GLOBAL['season_cliff']
    normalize = GLOBAL['normalize']
    last_game = compute_averages('last-game',
        weekly_alpha=weekly_alpha,
        season_cliff=season_cliff, normalize=normalize)
    this_season = compute_averages('this-season',
        weekly_alpha=weekly_alpha,
        season_cliff=season_cliff, normalize=normalize)
    last_season = compute_averages('last-season',
        weekly_alpha=weekly_alpha,
        season_cliff=season_cliff, normalize=normalize)

if not definition_only:
    Part_X()

Processing rows (X):  12%|██▉                    | 4/32 [00:00<00:00, 39.26it/s]

{'score', 'rushing-yard-ratio', 'redzone_yards', 'rushing-to-passing', 'att', 'sack_count', 'drives', 'elo', 'possession', 'passing-yards', 'fourth-down-conversion', 'third-down-conversion', 'redzone_count', 'comp-att', 'rushing-yards', 'def-st-td', 'sack_yards', 'fumbles', 'comp', 'rushing-attempts', 'third-downs', 'wintieloss', 'total-yards', 'passing-yard-ratio', 'espn-qbr', 'penalty_count', 'first-downs', 'fourth-downs', 'int', 'espn-qbr-missing', 'penalty_yards', 'turnovers', 'qb-elo'}


Processing rows (X):   6%|█▍                     | 2/32 [00:00<00:02, 11.89it/s]

{'score', 'rushing-yard-ratio', 'redzone_yards', 'rushing-to-passing', 'att', 'sack_count', 'drives', 'elo', 'possession', 'passing-yards', 'fourth-down-conversion', 'third-down-conversion', 'redzone_count', 'comp-att', 'rushing-yards', 'def-st-td', 'sack_yards', 'fumbles', 'comp', 'rushing-attempts', 'third-downs', 'wintieloss', 'total-yards', 'passing-yard-ratio', 'espn-qbr', 'penalty_count', 'first-downs', 'fourth-downs', 'int', 'espn-qbr-missing', 'penalty_yards', 'turnovers', 'qb-elo'}


Processing rows (X):   0%|                               | 0/32 [00:00<?, ?it/s]

{'score', 'rushing-yard-ratio', 'redzone_yards', 'rushing-to-passing', 'att', 'sack_count', 'drives', 'elo', 'possession', 'passing-yards', 'fourth-down-conversion', 'third-down-conversion', 'redzone_count', 'comp-att', 'rushing-yards', 'def-st-td', 'sack_yards', 'fumbles', 'comp', 'rushing-attempts', 'third-downs', 'wintieloss', 'total-yards', 'passing-yard-ratio', 'espn-qbr', 'penalty_count', 'first-downs', 'fourth-downs', 'int', 'espn-qbr-missing', 'penalty_yards', 'turnovers', 'qb-elo'}


Processing rows (X): 100%|██████████████████████| 32/32 [00:06<00:00,  5.17it/s]


# Part XI -- Final Data Frame

In [21]:
def make_table(averages_collection):
    
    excluded_keys = ['date', 'team', 'opponent', 'rivalry', 'home?']
    final = {}
    for index, r in tqdm(team_stat.iterrows(), total=len(team_stat), desc='Processing rows (XI)'):
        date = r['date']
        home = r['home']
        away = r['away']
        home_key = f'{home}|{date}'
        away_key = f'{away}|{date}'
        label = 1 if int(r['score_home']) >= int(r['score_away']) else 0
            # label is 1 if the home team win or tie, or 0 otherwise
        merged = {
            'date': date,
            'home': home,
            'away': away,
            'rivalry': f'{home}-{away}',
            'label': label
        }
        final_key = f'{home}-{away}|{date}'
        for (averages, name) in averages_collection:
            home_suffix = f'-home-{name}'
            away_suffix = f'-away-{name}'
            assert(home_key in averages and away_key in averages)
            home_entry = averages[home_key]
            home_entry = { k + home_suffix: v for k, v in home_entry.items() if k not in excluded_keys}
            away_entry = averages[away_key]
            away_entry = { k + away_suffix: v for k, v in away_entry.items() if k not in excluded_keys }
            
            merged.update(home_entry)
            merged.update(away_entry)

        final[final_key] = merged
            
    return final

In [22]:
def Part_XI():
    
    global tabular_dict, tabular_df
    
    averages_collection = [  # will turn this into features
        (last_game, 'last'),
        (this_season, 'season'),
        (last_season, 'last-season'),
    ]
    tabular_dict = make_table(averages_collection)    
    tabular_df = pd.DataFrame([v for k, v in tabular_dict.items()])
    tabular_df.to_csv(GLOBAL['filename'])
    print(f"exported to {GLOBAL['filename']}")
    
if not definition_only:
    Part_XI()

Processing rows (XI): 100%|███████████████| 4573/4573 [00:01<00:00, 3552.89it/s]


exported to tabular-095-050-norm.csv


In [23]:
def Part_All():
    parts = [
        Part_I,
        Part_II,
        Part_III,
        Part_IV,
        Part_V,
        Part_VI,
        Part_VII,
        Part_VIII,
        Part_IX,
        Part_X,
        Part_XI,
    ]
    for part in parts:
        part()

In [24]:
# Part_All()